# Working with Blocks

This notebook covers working with blocks in Notion - the building blocks of page content:
- Retrieving blocks from a page
- Appending different block types (paragraphs, headings, lists, code, etc.)
- Creating blocks with rich text formatting
- Updating and deleting blocks
- Working with nested blocks (toggles, nested lists)
- Building complete document structures

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID where we can create test content

## Setup: Load Dependencies and Initialize Client

In [1]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.1.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.models.blocks.Block
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.delay

// Initialize the client
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val parentPageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")

✅ NotionClient initialized successfully!


## Understanding Blocks in Notion

Blocks are the fundamental units of content in Notion. Every piece of content on a page is a block:
- **Text Blocks**: Paragraphs, headings, quotes, callouts
- **List Blocks**: Bulleted lists, numbered lists, to-do lists
- **Media Blocks**: Images, videos, files, code blocks
- **Layout Blocks**: Dividers, toggles, columns
- **Database Blocks**: Inline databases and linked views

In this notebook, we'll explore how to work with blocks programmatically.

## Example 1: Create a Test Page with Initial Content

First, let's create a test page with some initial blocks that we'll work with throughout this notebook.

In [2]:
val testPage = runBlocking {
    notion.pages.create {
        parent.page(parentPageId)
        title("Blocks Examples - Test Page")

        content {
            heading1("Welcome to Blocks!")
            paragraph("This page demonstrates working with blocks programmatically.")
        }

        icon.emoji("📦")
    }
}

val testPageId = testPage.id

println("✅ Test page created successfully!")
println("   Page ID: $testPageId")
println("   URL: ${testPage.url}")

// Wait a moment for the page to be fully created
runBlocking { delay(1000) }

✅ Test page created successfully!
   Page ID: 28fc63fd-82ed-811e-a4ea-e95466ade819
   URL: https://www.notion.so/Blocks-Examples-Test-Page-28fc63fd82ed811ea4eae95466ade819


## Example 2: Retrieve Block Children

Let's retrieve the blocks we just created and see what's on the page.

In [3]:
val blocks = runBlocking {
    notion.blocks.retrieveChildren(testPageId)
}

println("✅ Retrieved ${blocks.size} blocks from the page:")
blocks.forEach { block ->
    when (block) {
        is Block.Heading1 -> {
            val text = block.heading1.richText.firstOrNull()?.plainText
            println("   📌 Heading 1: $text")
        }
        is Block.Paragraph -> {
            val text = block.paragraph.richText.firstOrNull()?.plainText
            println("   📝 Paragraph: $text")
        }
        else -> println("   ❓ ${block.type}: ${block.id}")
    }
}

✅ Retrieved 2 blocks from the page:
   📌 Heading 1: Welcome to Blocks!
   📝 Paragraph: This page demonstrates working with blocks programmatically.


## Example 3: Append Simple Text Blocks

Let's add various types of text blocks to our page.

In [4]:
runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading1("Project Documentation")
        
        heading2("Overview")
        paragraph("This section provides an overview of the project.")
        
        heading3("Key Features")
        paragraph("Our project includes the following features...")
        
        divider()
        
        quote("The best way to predict the future is to invent it. - Alan Kay")
        
        divider()
    }
    
    delay(1000) // Wait for blocks to be created
}

println("✅ Text blocks appended successfully!")

✅ Text blocks appended successfully!


## Example 4: Append Blocks with Rich Text Formatting

Now let's create blocks with formatted text using the Rich Text DSL.

In [7]:
runBlocking {
    notion.blocks.appendChildren(testPageId) {
        paragraph {
            text("This paragraph contains ")
            bold("bold text")
            text(", ")
            italic("italic text")
            text(", ")
            code("inline code")
            text(", and ")
            strikethrough("strikethrough text")
            text(".")
        }
        
        paragraph {
            text("You can also combine formatting: ")
            boldItalic("bold + italic")
            text(", or use ")
            formattedText("multiple formats at once", bold = true, underline = true, italic = true)
            text(".")
        }
    }
    
    delay(1000)
}

println("✅ Formatted text blocks appended successfully!")

✅ Formatted text blocks appended successfully!


## Example 5: Create Different List Types

Notion supports three types of lists: bulleted lists, numbered lists, and to-do lists.

In [8]:
runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading2("Features Checklist")
        
        toDo("Set up development environment", checked = true)
        toDo("Implement core API client", checked = true)
        toDo("Write comprehensive tests", checked = false)
        toDo("Create documentation", checked = false)
        toDo("Deploy to production", checked = false)
        
        heading2("Technology Stack")
        
        bullet("Kotlin - Modern JVM language")
        bullet("Ktor - HTTP client library")
        bullet("Kotlinx.serialization - JSON handling")
        bullet("Kotest - Testing framework")
        
        heading2("Implementation Steps")
        
        number("Design API client architecture")
        number("Implement request/response models")
        number("Add authentication layer")
        number("Build DSL for common operations")
        number("Write unit and integration tests")
    }
    
    delay(1000)
}

println("✅ List blocks created successfully!")

✅ List blocks created successfully!


## Example 6: Create Code Blocks

Code blocks are perfect for displaying code. You can specify the language, though note that the Notion API doesn't return syntax-highlighted code (Notion applies highlighting in the UI, not in the API data).

In [ ]:
runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading2("Code Examples")
        
        paragraph("Here's how to initialize the Notion client in Kotlin:")
        
        code(
            language = "kotlin",
            code = """
                val notion = NotionClient(apiToken = "secret_...")
                val page = notion.pages.retrieve("page-id")
                println(page.title)
            """.trimIndent()
        )
        
        paragraph("And here's a Python example:")
        
        code(
            language = "python",
            code = """
                from notion_client import Client
                
                notion = Client(auth="secret_...")
                page = notion.pages.retrieve("page-id")
                print(page['properties']['title'])
            """.trimIndent()
        )
        
        callout("💡") {
            text("Note: The ")
            code("language")
            text(" parameter sets the language for the block (visible in Notion UI), but the API returns plain text without syntax highlighting.")
        }
    }
    
    delay(1000)
}

println("✅ Code blocks created successfully!")

## Example 7: Create Callout Blocks

Callouts are great for highlighting important information with an icon.

In [10]:
runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading2("Important Notes")
        
        callout("💡") {
            text("Tip: Use the Rich Text DSL for formatting text within blocks.")
        }
        
        callout("⚠️") {
            text("Warning: Always handle API rate limits appropriately.")
        }
        
        callout("✅") {
            text("Success: Your Notion integration is working correctly!")
        }
        
        callout("📚") {
            bold("Documentation: ")
            text("Check out the official Notion API docs at ")
            code("developers.notion.com")
        }
    }
    
    delay(1000)
}

println("✅ Callout blocks created successfully!")

✅ Callout blocks created successfully!


## Example 8: Work with Toggle Blocks

Toggle blocks allow you to create collapsible sections with nested content.

In [11]:
// Create a toggle block first
val result = runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading2("FAQ Section")
        toggle("What is Notion?")
    }
}

val toggleBlock = result.results.last()

runBlocking { delay(1000) }

// Add nested content inside the toggle
runBlocking {
    notion.blocks.appendChildren(toggleBlock.id) {
        paragraph("Notion is an all-in-one workspace for notes, tasks, wikis, and databases.")
        bullet("Flexible content organization")
        bullet("Collaborative features")
        bullet("Powerful API for automation")
    }
    
    delay(1000)
}

println("✅ Toggle block with nested content created successfully!")
println("   Toggle ID: ${toggleBlock.id}")

✅ Toggle block with nested content created successfully!
   Toggle ID: 28fc63fd-82ed-8166-834c-e0cb66f8460e


## Example 9: Update a Block

You can update existing blocks by their ID.

In [12]:
// First, create a block to update
val createResult = runBlocking {
    notion.blocks.appendChildren(testPageId) {
        heading2("Update Example")
        paragraph("This is the original text that will be updated.")
    }
}

val blockToUpdate = createResult.results.last()

runBlocking { delay(1000) }

// Now update it
val updatedBlock = runBlocking {
    notion.blocks.update(blockToUpdate.id) {
        paragraph {
            bold("This text has been updated!")
            text(" The update happened at ${java.time.LocalDateTime.now()}.")
        }
    }
}

println("✅ Block updated successfully!")
println("   Block ID: ${updatedBlock.id}")

val paragraphBlock = updatedBlock as? Block.Paragraph
val updatedText = paragraphBlock?.paragraph?.richText?.firstOrNull()?.plainText
println("   New text: $updatedText")

✅ Block updated successfully!
   Block ID: 28fc63fd-82ed-81cf-9f4d-e98f26f32a2a
   New text: This text has been updated!


## Example 10: Delete a Block

Blocks can be deleted (archived) by their ID.

In [13]:
// Create a block to delete
val deleteResult = runBlocking {
    notion.blocks.appendChildren(testPageId) {
        paragraph("This block will be deleted shortly.")
    }
}

val blockToDelete = deleteResult.results.first()

runBlocking { delay(1000) }

// Delete (archive) the block
val deletedBlock = runBlocking {
    notion.blocks.delete(blockToDelete.id)
}

println("✅ Block deleted successfully!")
println("   Block ID: ${deletedBlock.id}")
println("   Archived: ${deletedBlock.archived}")

✅ Block deleted successfully!
   Block ID: 28fc63fd-82ed-8164-a84e-fbcb34c9197b
   Archived: true


## Example 11: Create a Complete Document Structure

Let's put it all together and create a new page with a comprehensive document structure.

In [14]:
val documentPage = runBlocking {
    notion.pages.create {
        parent.page(parentPageId)
        title("Complete API Documentation Example")

        content {
            heading1("API Documentation")
            
            paragraph {
                text("Welcome to the ")
                bold("Kotlin Notion Client")
                text(" documentation. This page demonstrates a complete document structure.")
            }
            
            divider()
            
            heading2("Getting Started")
            
            paragraph("First, install the library from Maven Central:")
            
            code(
                language = "kotlin",
                code = """
                    dependencies {
                        implementation("it.saabel:kotlin-notion-client:0.1.0")
                    }
                """.trimIndent()
            )
            
            paragraph("Then initialize the client:")
            
            code(
                language = "kotlin",
                code = """
                    val notion = NotionClient(apiToken = "secret_...")
                """.trimIndent()
            )
            
            heading2("Key Features")
            
            bullet("Type-safe Kotlin API with DSL builders")
            bullet("Coroutine support for non-blocking I/O")
            bullet("Comprehensive error handling")
            bullet("Rich text formatting DSL")
            
            heading2("Basic Operations")
            
            callout("💡") {
                text("All API operations are ")
                code("suspend")
                text(" functions. Make sure to call them from a coroutine context.")
            }
            
            heading3("Pages")
            
            number("Retrieve a page by ID")
            number("Create new pages with properties")
            number("Update existing pages")
            number("Archive pages")
            
            heading3("Blocks")
            
            number("Append blocks to pages")
            number("Retrieve block children")
            number("Update block content")
            number("Delete blocks")
            
            divider()
            
            heading2("Best Practices")
            
            toDo("Always handle rate limits", checked = true)
            toDo("Use type-safe property access", checked = true)
            toDo("Leverage Kotlin coroutines for concurrent operations", checked = true)
            toDo("Include proper error handling", checked = false)
            
            divider()
            
            quote("Well-documented code is as valuable as the code itself.")
        }

        icon.emoji("📚")
    }
}

println("✅ Complete document created successfully!")
println("   Page ID: ${documentPage.id}")
println("   URL: ${documentPage.url}")

✅ Complete document created successfully!
   Page ID: 28fc63fd-82ed-8109-a4b9-db62397fdad6
   URL: https://www.notion.so/Complete-API-Documentation-Example-28fc63fd82ed8109a4b9db62397fdad6


## Cleanup

Let's clean up by archiving the test pages we created.

In [15]:
runBlocking {
    // Archive the test pages
    notion.pages.archive(testPageId)
    notion.pages.archive(documentPage.id)
}

println("✅ Test pages archived successfully!")

✅ Test pages archived successfully!


## Summary

In this notebook, we explored working with blocks in Notion:

### Key Operations
- **Retrieve**: Get blocks from a page using `blocks.retrieveChildren(pageId)`
- **Append**: Add new blocks using `blocks.appendChildren(blockId) { ... }`
- **Update**: Modify existing blocks using `blocks.update(blockId) { ... }`
- **Delete**: Archive blocks using `blocks.delete(blockId)`

### Block Types Covered
- **Text**: Paragraphs, headings (H1, H2, H3), quotes
- **Lists**: Bulleted lists, numbered lists, to-do lists
- **Code**: Code blocks with syntax highlighting
- **Callouts**: Highlighted boxes with icons
- **Layout**: Dividers, toggles (collapsible sections)

### Rich Text Formatting
- Use the Rich Text DSL within blocks for formatting
- Supports: **bold**, *italic*, `code`, ~~strikethrough~~, underline
- Combine multiple formats for complex styling

### Best Practices
1. Use `runBlocking` or other coroutine contexts for suspend functions
2. Add `delay()` after mutations to allow API changes to propagate
3. Use type-safe block access with pattern matching (`as? Block.Paragraph`)
4. Nest blocks by appending children to a parent block's ID
5. Build complete documents with the `content { }` DSL when creating pages

## Next Steps

Explore more advanced topics:
- **[05-rich-text-dsl.ipynb](./05-rich-text-dsl.ipynb)** - Deep dive into text formatting
- **[06-advanced-queries.ipynb](./06-advanced-queries.ipynb)** - Complex filtering and pagination